In [44]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
import os
from langgraph.prebuilt import ToolNode
from typing import TypedDict, Annotated, Literal
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage

load_dotenv()

True

In [45]:
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
os.environ['TAVILY_API_KEY'] = os.getenv('TAVILY_API_KEY')
os.environ['LANGCHAIN_TRACING_V2'] = os.getenv('LANGCHAIN_TRACING_V2')
os.environ['LANGCHAIN_ENDPOINT'] = os.getenv('LANGCHAIN_ENDPOINT')
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')

In [46]:
tools = [TavilySearchResults(max_results=1)]

In [47]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, streaming=True)

In [48]:
model = model.bind_tools(tools)

In [49]:
tool_node = ToolNode(tools)

In [50]:
def add_messages(left: list, right: list):
    """Añadir sin sobrescribir."""
    return left + right

In [51]:
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

In [52]:
# Definir la función que determina si continuar o no
def should_continue(state: AgentState) -> Literal["tools", "__end__"]:
    messages = state['messages']
    last_message = messages[-1]
    # Si el LLM hace una llamada a una herramienta, entonces enrutamos al nodo "tools"
    if last_message.tool_calls:
        return "tools"
    # De lo contrario, nos detenemos (respondemos al usuario)
    return "__end__"

In [53]:
# Definir la función que llama al modelo
def call_model(state: AgentState):
    messages = state['messages']
    
    response = model.invoke(messages)
 
    # Devolvemos una lista, porque esto se agregará a la lista existente
    return {"messages": [response]}

## Definir el Grafo

In [54]:
# Definir un nuevo grafo
workflow = StateGraph(AgentState)


In [55]:
# Definir los dos nodos entre los que ciclará
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

In [56]:
# Establecer el punto de entrada como `agent`
workflow.set_entry_point("agent")

In [57]:
# Agregar un borde condicional
workflow.add_conditional_edges(
    "agent",
    should_continue,
)

In [58]:
# Agregar un borde normal de `tools` a `agent`
workflow.add_edge('tools', 'agent')

In [59]:
# Finalmente, ¡compilamos!
app = workflow.compile()

In [60]:
mensaje = HumanMessage(content="¿Cuál es el clima en San Francisco? Contesta en español")

In [61]:
inputs = {"messages": [mensaje]}
inputs


{'messages': [HumanMessage(content='¿Cuál es el clima en San Francisco? Contesta en español')]}

In [62]:
for output in app.stream(inputs, stream_mode="updates"):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Salida del Nodo '{key}':")
        print("---")
        print(value)
    print("\n---\n")

Salida del Nodo 'agent':
---
{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_QTK7gdFGDJjtLzdRjkNuk832', 'function': {'arguments': '{"query":"clima en San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-44bd3ea7-1b22-4762-afc3-462af1d7585c-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'clima en San Francisco'}, 'id': 'call_QTK7gdFGDJjtLzdRjkNuk832'}])]}

---

Salida del Nodo 'tools':
---
{'messages': [ToolMessage(content='[{"url": "https://weather.com/es-US/tiempo/10dias/l/San+Francisco+CA+USCA0987:1:US/", "content": "Prep\\u00e1rate con el pron\\u00f3stico para los pr\\u00f3ximos 10 d\\u00edas m\\u00e1s preciso para San Francisco, CA. Consulta la temperatura m\\u00e1xima y m\\u00ednima y la probabilidad de lluvia en The Weather Channel y ..."}]', name='tavily_search_results_json', tool_call_id='call_QTK7gdFGDJjtLzdRjkNuk832')]}